# Formato para la implementación de los ejercicios de la Tarea  2 en Python

### Los programas implementados para la  solución de los ejercicios se evaluarán ejecutando una notebook con el formato que se describe a continuación. Cada estudiante debe asegurarse que sus programas se ejecutan de manera correcta y en un tiempo razonable utilizando este formato. 

### Los programas se pueden implementar en Python ó R. No deben implementarse en los dos lenguajes pues solamente se evaluará en uno de ellos. El código en Python o R podrá incluirse directamente en las celdas de la notebook o implementarse independientemente para ser invocado DESDE LA NOTEBOOK. Es decir, los programas no se evaluarán independientemente sino a partir del llamado que se hace en la notebook. 

In [1]:
# Ejemplo de resolución del problema QAP en Python
# Importamos las librerías imprescindibles para la ejecución del ejercicio
import numpy as np

In [2]:
def ReadQAPInstance(fname):
    # Este programa recibe el nombre y ubicación de un fichero con los datos de una instancia del problema QAP
    # y devuelve una estructura con los datos de la instancia.
    hdl = open(fname, 'r')
    mylist = hdl.readlines()
    hdl.close()
    n = eval(mylist[0])

    w  = np.zeros((n,n))
    for i in range(n):
        for j,val in enumerate(mylist[i+1].split()):     
            w[i,j]=eval(val)
    
    d  = np.zeros((n,n))
    for i in range(n,2*n):
        for j,val in enumerate(mylist[i+1].split()):     
            d[i-n,j]=eval(val)
    
    QAPInstance = { "weights" : w,
                   "distances" : d}

    return QAPInstance

In [3]:
def QAPEvaluator( Dist_Matrix, Weigth_Matrix, permutation):
    n = Dist_Matrix.shape[0]   
    perm = np.asarray(permutation) - 1                           # La representación en python comienza en cero   
    val_qap = 0  # 
    for i in range(n):
        for j in range(n):    
            val_qap = val_qap + Weigth_Matrix[i,j] * Dist_Matrix[perm[i],perm[j]]# Coste utlizacion entre instalaciones consecutivas   
  
    # Finalmente se devuelve el resultado
    return val_qap  

In [4]:
#my_QAP_Instance = ReadQAPInstance('../Instances/QAP/Cebe.qap.n10.1')  
  
#Dist_Matrix     = my_QAP_Instance["distances"]
#Weigth_Matrix     = my_QAP_Instance["weights"]

#print(np.shape(Dist_Matrix))
#print(np.shape(Weigth_Matrix))

#QAPEvaluator( Dist_Matrix, Weigth_Matrix, [1,3,2,4,10,9,8,7,6,5])

In [5]:
# Swap crea una vecindad basada en el operador de intercambio entre posiciones
# Todas las permutaciones que se pueden obtener como un swap entre la primera posición
# y cualquiera de las restantes están en la vecindad
def getAllPairwiseComps(perm):
    n = perm.shape[0]
    n_neighbors = int( n*(n-1)/2 )           # Número de vecinos
    neighbors = np.zeros((n_neighbors,n),dtype=int) # Guardaremos todos los vecinos en neighbors 
    ind = 0
    for j in range(n):
        for i in range(j+1,n):
            neighbors[ind,:] = perm 
            neighbors[ind,i] = perm[j]
            neighbors[ind,j] = perm[i]  
            ind = ind + 1   
    return neighbors

In [6]:
#getAllPairwiseComps( np.array([1,2,3,4]) )

In [7]:
#n=10
#init_sol = np.random.permutation(n)+1
#print( init_sol )
#neighbors = getAllPairwiseComps(init_sol)
#print( neighbors.shape[0] )

In [8]:
def QAPLocalSearch(fname,permutation):
    best_val = 0
    init_sol = np.asarray( permutation )
    
    # IMPORTANTE:  Se lee la instancia  UNA SOLA VEZ.
    # ES INCORRECTO LEER LA INSTANCIA DEL FICHERO CADA VEZ QUE SE EVALUA UNA SOLUCIÓN
    my_QAP_Instance = ReadQAPInstance(fname)  
  
    Dist_Matrix       = my_QAP_Instance["distances"]
    Weigth_Matrix     = my_QAP_Instance["weights"]
    
    # En los siguientes pasos se implementa la evaluación la búsqueda local en Python   
    n = Dist_Matrix.shape[1]   
    best_val = QAPEvaluator( Dist_Matrix, Weigth_Matrix, init_sol )              # Mejor valor
    best_sol = init_sol                                             # Mejor solución 
    improvement = True
    number_evaluations = 1   
    while improvement:                    # Mientras se mejore el valor de la función
        neighbors = getAllPairwiseComps(best_sol)       # Todos los vecinos
        n_neighbors = neighbors.shape[0]
        number_evaluations =  number_evaluations + n_neighbors  # Se calcula es número de evaluaciones
        best_val_among_neighbors = best_val
        for i in range(n_neighbors):                    # Se recorren todos los vecinos buscando el mejor 
            sol = neighbors[i,:]  
            fval =  QAPEvaluator( Dist_Matrix, Weigth_Matrix, sol )    # Se evalua la función
            if fval<best_val_among_neighbors:             # Si es mejor que el mejor valor entre los vecinos hasta el momento
                best_val_among_neighbors = fval             # se actualiza el mejor valor
                best_sol_among_neighbors = sol   
        improvement = (best_val_among_neighbors<best_val) #  Se determina si ha habido mejora con respecto al ciclo anterior
        if improvement:                                
            best_val = best_val_among_neighbors           # Se actualiza el mejor valor y la mejor solución 
            best_sol = best_sol_among_neighbors      
            #print(best_val,best_sol, number_evaluations)
        if (number_evaluations > 1e6):
            print("Iterations > 1e6. Aborting local search.")
            break

    # Finalmente se devuelven el mejor valor encontrado así como la mejor solución
    return best_val,best_sol       

In [9]:
# En esta parte comprobamos la implementación de los programas. 
# Esta celda no debe ser modificada

# Evaluación del algoritmo para 5 instancias de tamaño 10
filename_base = '../Instances/QAP/Cebe.qap.n10.'
mypermutation_10 = [1,3,2,4,10,9,8,7,6,5]
for i in range(5):
  fname  = filename_base+str(i+1)
  best_val,best_sol = QAPLocalSearch(fname,mypermutation_10)  
  print(10,i+1,best_val,best_sol)  
  

# Evaluación del algoritmo para 5 instancias de tamaño 20
filename_base = '../Instances/QAP/Cebe.qap.n20.'
mypermutation_20 = range(1,21)
for i in range(5):
  fname  = filename_base+str(i+1)
  best_val,best_sol = QAPLocalSearch(fname,mypermutation_20)  
  print(20,i+1,best_val,best_sol)  


# Evaluación del algoritmo para 5 instancias de tamaño 30
filename_base = '../Instances/QAP/Cebe.qap.n30.'
mypermutation_30 = range(1,31)
for i in range(5):
  fname  = filename_base+str(i+1)
  best_val,best_sol = QAPLocalSearch(fname,mypermutation_30)  
  print(30,i+1,best_val,best_sol)  

FileNotFoundError: [Errno 2] No such file or directory: '../Instances/QAP/Cebe.qap.n10.1'

In [ ]:
# Ejemplo de la implementación de la búsqueda local para el problema Bipartitioning en Python

# Importamos las librerías imprescindibles para la ejecución del ejercicio
import numpy as np

In [ ]:
def Read_Bip_Instance(fname):
    # Este programa recibe el nombre y ubicación de un fichero con los datos de una instancia del problema BiPartioning
    # y devuelve una estructura con los datos de la instancia.
    hdl = open(fname, 'r')           
    mylist = hdl.readlines()
    hdl.close()
    n = eval(mylist[0])      
    edge_weights = np.zeros((n,n))      # Pesos de las aristas 
    for i in range(n):
        for j,val in enumerate(mylist[i+1].split()):     
            edge_weights[i,j]=eval(val) 
    
    BipPartInstance = {"edge_weights":edge_weights}

    return BipPartInstance

In [ ]:
def BipEvaluator(edge_weights,solution):
    n = edge_weights.shape[0]         # Número de nodos
    balance =  np.sum(solution) # Numero de nodos en una de las partes
    fval = 0                  # Peso de las aristas entre partes del grafo
    for i in range(n-1):
        for j in range(i+1,n):
            if solution[i]==1-solution[j]:      # Si estan en partes diferentes  
                fval = fval+edge_weights[i,j]  

    feasible=(balance==n/2)            
    # Finalmente se devuelve el resultado
    return feasible,fval,balance

In [ ]:
# TwoOpt crea una vecindad basda en el operador two-opt de forma determinista
# Todas las permutaciones que se pueden obtener con two-opt están en la vecindad
def SwapAll1AND0(perm):
    n = perm.shape[0]
    ix1 = np.where( perm == 1)[0]
    ix0 = np.where( perm == 0)[0]
    n_neighbors = len(ix1)*len(ix0)  # Número de vecinos    
    neighbors = np.zeros((n_neighbors,n),dtype=int) # Guardaremos todos los vecinos en neighbors
    ind = 0
    for i in range(len(ix1)):
        for j in range(len(ix0)):
            neighbors[ind,:] = perm   
            aux = perm.copy() 
            neighbors[ind,ix1[i]] = aux[ix0[j]]   # Se invierte el camino entre posiciones elegidas
            neighbors[ind,ix0[j]] = aux[ix1[i]]   # Se invierte el camino entre posiciones elegidas
            ind = ind + 1   
    return neighbors

In [ ]:
#SwapAll1AND0( np.array([1,1,0,0]) )

In [ ]:
#my_solution_10 = np.hstack((np.ones((5)),np.zeros((5))))
#print( my_solution_10 )

#SwapAll1AND0( my_solution_10 )

In [ ]:
#my_BipPart_Instance = Read_Bip_Instance('../Instances/BIPART/Cebe.bip.n10.1')  # Se lee la instancia
#edge_weights = my_BipPart_Instance["edge_weights"]
#print( np.shape(edge_weights) )

In [ ]:
def BipPartLocalSearch(fname,solution):
    best_val = []
    init_sol = solution

    my_BipPart_Instance = Read_Bip_Instance(fname)  # Se lee la instancia
    edge_weights = my_BipPart_Instance["edge_weights"]
    
    # En los siguientes pasos se implementa la búsqueda local para el problema BipPart 
    n = edge_weights.shape[1]   
    feasible,fval,balance = BipEvaluator( edge_weights, init_sol )              # Mejor valor
    best_val = fval
    best_sol = init_sol                                             # Mejor solución 
    improvement = True
    number_evaluations = 1   
    while improvement:                    # Mientras se mejore el valor de la función
        neighbors = SwapAll1AND0(best_sol)       # Todos los vecinos
        n_neighbors = neighbors.shape[0]
        number_evaluations =  number_evaluations + n_neighbors  # Se calcula es número de evaluaciones
        best_val_among_neighbors = best_val
        for i in range(n_neighbors):                    # Se recorren todos los vecinos buscando el mejor 
            sol = neighbors[i,:]  
            feasible,fval,balance =  BipEvaluator( edge_weights, sol )    # Se evalua la función
            if (feasible):
                if fval>best_val_among_neighbors:             # Si es mejor que el mejor valor entre los vecinos hasta el momento
                    best_val_among_neighbors = fval             # se actualiza el mejor valor
                    best_sol_among_neighbors = sol   
        improvement = (best_val_among_neighbors>best_val) #  Se determina si ha habido mejora con respecto al ciclo anterior
        if improvement:                                
            best_val = best_val_among_neighbors           # Se actualiza el mejor valor y la mejor solución 
            best_sol = best_sol_among_neighbors      
            #print(best_val,best_sol, number_evaluations)
        if (number_evaluations > 1e6):
            print("Iterations > 1e6. Aborting local search.")
            break
    # Finalmente se devuelven el mejor valor encontrado así como la mejor solución
    return best_val,best_sol        

In [ ]:
# En esta parte comprobamos la implementación de los programas. 
# Esta celda no debe ser modificada



# Evaluación del algoritmo para 5 instancias de tamaño n = 10
filename_base = '../Instances/BIPART/Cebe.bip.n10.'
my_solution_10 = np.hstack((np.ones((5)),np.zeros((5))))
for i in range(5):
  fname  = filename_base+str(i+1)
  best_val,best_sol = BipPartLocalSearch(fname,my_solution_10)  
  print(10,i+1,best_val,best_sol)  

# Evaluación del algoritmo para 5 instancias de tamaño n = 20
filename_base = '../Instances/BIPART/Cebe.bip.n20.'
my_solution_20 = np.hstack((np.ones((10)),np.zeros((10))))
for i in range(5):
  fname  = filename_base+str(i+1) 
  best_val,best_sol = BipPartLocalSearch(fname,my_solution_20)  
  print(20,i+1,best_val,best_sol)  


# Evaluación del algoritmo para 5 instancias de tamaño n = 50
filename_base = '../Instances/BIPART/Cebe.bip.n50.'
my_solution_50 = np.hstack((np.ones((25)),np.zeros((25))))
for i in range(5):
  fname  = filename_base+str(i+1) 
  best_val,best_sol = BipPartLocalSearch(fname,my_solution_50)  
  print(50,i+1,best_val,best_sol)  


# Evaluación del algoritmo para 5 instancias de tamaño n = 80
filename_base = '../Instances/BIPART/Cebe.bip.n80.'
my_solution_80 = np.hstack((np.ones((40)),np.zeros((40))))
for i in range(5):
  fname  = filename_base+str(i+1) 
  best_val,best_sol = BipPartLocalSearch(fname,my_solution_80)  
  print(80,i+1,best_val,best_sol)  